# Redis - Simple Social Network Datastore Example

This source code contains example of redis command based on the datastructure of simple social network.

In [1]:
import redis, csv

In [2]:
# connect to the local redis server

rd = redis.Redis()

In [3]:
# remove all data so that it is clear if we rerun this notebook

rd.flushall()

True

## Import data
Read data from csv files and store all data (user information and relationship) in the redis

In [4]:
with open('users.csv', encoding='utf-8') as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        id = row[0]
        name = row[1]
        k = 'user:{}:name'.format(id)
        rd.set(k, name)
        k = 'username:{}'.format(name)
        rd.set(k, id)

In [5]:
with open('follows.csv', encoding='utf-8') as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        id = row[0]
        follow = row[1]
        k = 'user:{}:follows'.format(id)
        rd.sadd(k, follow)
        k = 'user:{}:followed_by'.format(follow)
        rd.sadd(k, id)

## Data Retrieval
To retrieve data from redis, we will have to know the data structure being used for that particular key and use command accordingly.  In this case, user information is stored as "String" type and following information is stored as "Set"

In [6]:
id = 141
name = rd.get('user:{}:name'.format(id)).decode('utf-8')
name

'crushedGnu1'

In [7]:
rd.get('user:141:name').decode('utf-8')

'crushedGnu1'

In [8]:
rd.get('username:{}'.format(name)).decode('utf-8')

'141'

In [9]:
rd.smembers('user:{}:follows'.format(id))

{b'106', b'108', b'119', b'138', b'143', b'162', b'164', b'180'}

In [10]:
rd.smembers('user:{}:followed_by'.format(id))

{b'106', b'152', b'187'}

## Key scan
We can scan all keys in the database using "scan" command.  We call scan with cursor (initial at 0) to indicate the place to start scanning.  We also indicate the matching pattern to focus our scan to specific key pattern.  Each call returns a tuple of (next_cursor, results).  We can use next_cursor to continue our scanning process.

In [16]:
cursor = 0
cursor, keys = rd.scan(cursor=cursor, match='username:*')
while cursor > 0:
    for x in keys:
        key = x.decode('utf-8')
        value = rd.get(key).decode('utf-8')
        print(key, value)
        
    cursor, keys = rd.scan(cursor=cursor, match='username:*')  

username:abjectSalt5 145
username:peskyPiglet5 190
username:mercifulPlover5 156
username:debonairBobolink7 125
username:trustingIguana7 135
username:solidOil8 172
username:cautiousTuna9 195
username:bubblyWigeon3 183
username:wakefulTomatoe3 169
username:solemnOryx8 152
username:lyingThrushe1 111
username:pitifulSalami5 134
username:adoringGatorade6 200
username:needfulRuffs7 131
username:madTeal5 163
username:solemnRelish2 174
username:outlyingMeerkat2 170
username:guiltyMeerkat7 138
username:grumpyGatorade9 144
username:betrayedEagle8 194
username:cruelStork3 123
username:aboardMuesli8 154
username:pitifulShads3 116
username:jumpySheep0 113
username:thrilledPonie5 129
username:ferventCur8 142
username:artisticWildfowl3 155
username:similarCow8 139
username:dejectedOil5 157
username:similarWasp4 159
username:enragedDoves7 176
username:contentMandrill3 173
username:bubblyZebra1 114
username:dreadfulAbalone4 110
username:ecstaticTacos2 188
username:solemnBoars0 179
username:wrathfulElan

In [14]:
rd.scan(0)

(96,
 [b'username:abjectSalt5',
  b'user:129:name',
  b'user:120:followed_by',
  b'user:101:follows',
  b'user:187:followed_by',
  b'user:122:name',
  b'user:153:follows',
  b'username:peskyPiglet5',
  b'user:117:name',
  b'user:176:followed_by'])

In [15]:
rd.scan(96)

(304,
 [b'user:142:name',
  b'username:mercifulPlover5',
  b'user:184:followed_by',
  b'user:127:followed_by',
  b'user:145:name',
  b'user:184:follows',
  b'user:166:name',
  b'username:debonairBobolink7',
  b'user:163:name',
  b'username:trustingIguana7',
  b'user:141:follows',
  b'user:164:followed_by'])